# Exploring Window Functions with SQL and Pandas
Based on http://www.helenanderson.co.nz/sql-window-functions-part-1/

In [1]:
import numpy as np
import pandas as pd
np.random.seed(20190904)

/Users/sinayoks/apps/miniconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1066: UserWarning: Duplicate key in file "/Users/sinayoks/.matplotlib/matplotlibrc", line #381
  (fname, cnt))


In [2]:
import psycopg2
import psycopg2.extras



# Generate sample dataset

In [3]:
N = 1000
NB_YEARS = 1
START_DATE = '2018-01-01'

In [4]:
def sample_dataset(start_date=START_DATE, nb_years=NB_YEARS, size=N):
    ordermonth = pd.Series(
        pd.to_datetime(
            np.random.randint(0, 365 * nb_years, size=size), 
            unit='D', 
            origin=start_date
        )
    ).dt.strftime('%Y-%m')
    territoryid = pd.Series(np.random.randint(0, 10, size))
    subtotal = pd.Series(np.random.uniform(500, 2500, size=size))
    taxamt = 0.075 * subtotal
    freight = 0.025 * subtotal
    totaldue = subtotal + taxamt
    customerid = pd.Series(np.random.randint(1, size//10 + 1, size) + 2000)
    salesorderid = np.arange(4000, 4000 + size)
    df = pd.DataFrame({
        'salesorderid': salesorderid,
        'ordermonth': ordermonth,
        'territoryid': territoryid,
        'subtotal': subtotal,
        'taxamt': taxamt,
        'freight': freight,
        'totaldue': totaldue,
        'customerid': customerid
    })
    return df

In [5]:
dataset = sample_dataset()
dataset.head()

,salesorderid,ordermonth,territoryid,subtotal,taxamt,freight,totaldue,customerid
0,4000,2018-10,2,1867.940043,140.095503,46.698501,2008.035546,2079
1,4001,2018-08,7,1037.805109,77.835383,25.945128,1115.640492,2058
2,4002,2018-08,3,2318.164763,173.862357,57.954119,2492.027120,2016
3,4003,2018-08,1,2104.229867,157.817240,52.605747,2262.047107,2042
4,4004,2018-03,6,2370.179348,177.763451,59.254484,2547.942799,2067


In [6]:
dataset.to_csv('sales.csv', index=False)

In [7]:
pwd

'/Users/sinayoks/dev/sql'

In [8]:
!head sales.csv

salesorderid,ordermonth,territoryid,subtotal,taxamt,freight,totaldue,customerid
4000,2018-10,2,1867.9400426683374,140.0955032001253,46.69850106670844,2008.0355458684626,2079
4001,2018-08,7,1037.8051087290623,77.83538315467968,25.94512771822656,1115.640491883742,2058
4002,2018-08,3,2318.1647625406504,173.86235719054878,57.95411906351626,2492.027119731199,2016
4003,2018-08,1,2104.22986723532,157.817240042649,52.605746680883,2262.0471072779687,2042
4004,2018-03,6,2370.179347712942,177.76345107847067,59.254483692823555,2547.9427987914128,2067
4005,2018-05,8,1078.934365009996,80.9200773757497,26.973359125249903,1159.8544423857459,2055
4006,2018-07,5,900.3227133346401,67.524203500098,22.508067833366002,967.8469168347381,2057
4007,2018-05,5,1720.0294820306863,129.00221115230147,43.00073705076716,1849.0316931829877,2050
4008,2018-11,7,994.5347728977076,74.59010796732807,24.863369322442693,1069.1248808650357,2096


# create postgres database

In [9]:
def connect_db(dbname, user=None, **kwargs):
    user = user or 'postgres'
    con = psycopg2.connect(database=dbname, user=user, **kwargs)
    con.autocommit = True
    return con

In [10]:
def create_db(dbname='sales', drop=True):
    con = connect_db('postgres')
    cur = con.cursor()
    cur.execute(f'DROP DATABASE IF EXISTS {dbname};')
    cur.execute(f'DROP USER IF EXISTS {dbname};')
    cur.execute(f'CREATE DATABASE {dbname};')
    cur.execute(f'CREATE USER {dbname};')
    con.close()

def create_table(dbname='sales', filename='sales.csv'):
    """Import data from csv into postgres database.
    
    Uses psycopg2 and sql COPY. 
    Could also use pandas.to_sql but this removes dependency on pandas and is also more scaleable.
    """
    con = connect_db(dbname)
    with con.cursor() as cur:
        cur.execute(f'DROP TABLE IF EXISTS {dbname};')
        cur.execute(f"""
CREATE TABLE {dbname} (
salesorderid INTEGER UNIQUE NOT NULL,
ordermonth VARCHAR (7) NOT NULL,
territoryid FLOAT NOT NULL,
subtotal FLOAT NOT NULL,
taxamt FLOAT NOT NULL,
freight FLOAT NOT NULL,
totaldue FLOAT NOT NULL,
customerid INTEGER NOT NULL
);""")
    with con.cursor() as cur:
        cur.execute(
        f"COPY {dbname} FROM '/Users/sinayoks/dev/sql/{filename}' WITH (FORMAT csv, HEADER true);"
        )
        cur.execute(f'GRANT SELECT ON {dbname} TO {dbname};')
    con.close()

def query(query, dbname='sales', columns=dataset.columns):
    con = connect_db(dbname, user=dbname)
    with con.cursor(cursor_factory=psycopg2.extras.NamedTupleCursor) as cur:
        cur.execute(query)
        return cur.fetchall()

def queryframe(*args, **kwargs):
    data = query(*args, **kwargs)
    return pd.DataFrame(data)
    
    

In [11]:
create_db()

In [12]:
create_table()

In [13]:
query('select * from sales limit 2')

[Record(salesorderid=4000, ordermonth='2018-10', territoryid=2.0, subtotal=1867.94004266834, taxamt=140.095503200125, freight=46.6985010667084, totaldue=2008.03554586846, customerid=2079),
 Record(salesorderid=4001, ordermonth='2018-08', territoryid=7.0, subtotal=1037.80510872906, taxamt=77.8353831546797, freight=25.9451277182266, totaldue=1115.64049188374, customerid=2058)]

In [14]:
queryframe('select * from sales limit 2')

,salesorderid,ordermonth,territoryid,subtotal,taxamt,freight,totaldue,customerid
0,4000,2018-10,2.0,1867.940043,140.095503,46.698501,2008.035546,2079
1,4001,2018-08,7.0,1037.805109,77.835383,25.945128,1115.640492,2058
